# 第六章 页面置换算法

**功能**：当缺页中断发生，需要调入新的页面而内存已满时，选择内存当中哪个物理页面被置换。

**目标**：
* 尽可能地减少页面的换进换出。
* 页面锁定（Frame Locking）：用于描述必须常驻内存的操作系统的关键部分或时间关键（time-critical）的应用进程。它的实现方法为在页表中添加锁定标志位（lock bit）。

## 6.1 最优页面置换算法

**基本思路**：当一个缺页中断发生时，对于保存在内存当中的每一个逻辑页面，计算在它下一次访问之前还需等待多长时间，从中选择等待时间最长的那个座位被置换算法。

> Tips: 最优页面置换算法是理想情况，因此程序行为不可预知，该算法一般用作其他算法的性能评估依据。

## 6.2 先进先出算法

先进先出算法（First-In First-Out，FIFO）的**基本思路**是选择在内存中驻留时间最长的页面淘汰。

**实现思路**：系统维护着一个链表，记录当前内存中的逻辑页，链首页面的驻留时间最长，链尾页面的驻留时间最短。发生缺页时，把链首淘汰，并把新的页面添加到链尾。

**缺点**：性能较差，并有 Belady 现象。

## 6.3 最近最久未使用算法

最近最久未使用算法（Least Recently Used，LRU）的**基本思路**为当一个缺页中断时，选择最久未被使用的那个页进行淘汰。

**两种实现思路**：
* 系统维护一个页面链表，刚刚使用过的页面作为首结点，最久未使用的页面作为尾结点。
* 设置一个活动页面栈，当访问某个页时，将次页号压入栈顶，然后考察栈内是否有与此页面相同的页号，若有则删除；栈底的页面为最久未使用页面。

**缺点**：实现的开销大。

## 6.4 时钟页面置换算法

时钟页面置换算法（Clock）的**基本思路**为：
* 需要用到页表项当中的访问位，当一个页面被装入内存时，把该位初始化为0，然后如果这个页面被访问，则该位置为1。
* 把各个页面组织成环形链表，把指针指向最老的页面。
* 当发生缺页中断，考察当前指针所指向页面的访问位，若为0，则立即淘汰，若为1，则将其置为0，并移动到下一位。

## 6.5 二次机会法

二次机会法是时钟页面置换算法的一个改进，它的**基本思路**是基于访问位和修改位来决定淘汰页，如果当前指针所指向页面的：
* 访问位为0，修改位为0，则该页将被替换。
* 访问位为0，修改位为1，则将修改位置1，并将该页面写回磁盘。
* 访问位为1，修改位为0，则将访问位置0。
* 访问位为1，修改位为1，则将访问位置0。

## 6.6 最不常用算法

最不常用算法（Least Frequently Used，LFU）的**基本思路**为选择访问次数最少的页面进行淘汰。

**实现方法**：对每个页面设置一个访问计数器。

**问题**：一个页面在进程开始时使用的很多次，但随着程序执行，页面不需要被访问；算法的实现费力。

**解决办法**：定期把计数器位减少。

## 6.7 Belady 现象，LRU、FIFO、Clock的比较

**Belady现象**：在采用FIFO算法时，有时会出现分配的物理页面数增加，缺页率反而提高的异常现象。这种现象出现的原因在于算法的置换特征与进程访问内存的动态特征相矛盾。

**为什么LRU页面置换算法没有Belady？**：LRU符合栈算法的特点。

**LRU、FIFO、Clock算法的比较**：

* LRU算法和FIFO算法本质上都是先进先出的思路，只不过LRU是针对页面的最近访问顺序，而FIFO是针对页面进入内存的时间来进行排序。
* LRU算法开销大，FIFO算法开销小，但效果不好，因此采用折中的Clock算法。

## 6.8 局部页面置换算法的问题、工作集模型

**局部页面置换算法的问题**：单个进程所需的物理页帧随时间变化而改变。

**工作集模型**：

* 如果程序局部性原理不成立，那么各种页面置换算法就没有区别。
* 如果程序局部性原理成立，那么可以用工作集模型来对它进行定量地分析。

**工作集**：一个进程当前正在使用的逻辑页面集合，可以用一个二元函数$W(t,\delta)$表示。

* $t$ - 当前的执行时刻。
* $\Delta$ - 工作集窗口（Working-set window），即一个定长的页面访问的时间窗口。
* $W(t,\Delta)$ - 在当前时刻$t$之前的$\delta$时间窗口当中的所有页面所组成的集合。
* $|W(t,\Delta)|$ - 指工作集的大小，即页面数目。

**常驻集**：指在当前时刻，进程实际驻留在内存中的页面几何，常驻集 $\supseteq$ 工作集。当进程常驻集的大小达到某个数目之后，再给它分配更多的物理页面，缺页率也不会明显下降。

通过动态地变化每个进程的常驻集大小以适应它当前的工作集，从而使得系统整体的缺页率降低。

## 6.9 两个全局置换算法

**工作集置换算法**的基本思路为：当页面不属于工作集，则淘汰，而不是等到发生缺页中断时才淘汰（对于每个进程的常驻集不可变的情况）。

**可变分配策略**不同于工作集置换算法，它的每个进程的常驻集大小可变。它的具体实现是使用缺页率算法（Page Fault Frquency，PFF）来动态调整常驻集大小。**优缺点**：性能较好，但增加了系统开销。

**缺页率**：缺页次数/内存访问次数。它的影响因素有：
* 页面置换算法
* 常驻集大小。
* 页面本身的大小。
* 程序的编写方法。

若运行的程序缺页率过高，则通过增加工作集分配更多的物理页帧；若运行的程序缺页率低，则通过工作集来减少物理页帧。

## 6.10 抖动问题

如果分配给一个进程的物理页面太少，不能包含整个工作集，即常驻集$\sqsupset$，那么进程将会造成很多的缺页中断。

**抖动产生的原因**：随着驻留内存的进程数目增加，分配给每个进程的物理页面数不断减少，却也率不断上升。

**量化**：平均页缺失时间（Mean Time Between Page Fault，MTBF）和内存的大小。